Get Started with Tensor Expression



In [3]:
from __future__ import absolute_import, print_function

import tvm
import tvm.testing
from tvm import te
import numpy as np

# Global declarations of environment.

tgt_host = "llvm"


## Vector Add Example



In [4]:
n = te.var("n")
A = te.placeholder((n,), name="A")
B = te.placeholder((n,), name="B")
C = te.compute(A.shape, lambda i: A[i] + B[i], name="C")
print(type(C))

<class 'tvm.te.tensor.Tensor'>


In [5]:
s = te.create_schedule(C.op)

In [6]:
bx, tx = s[C].split(C.op.axis[0], factor=64)


In [14]:
fadd = tvm.build(s, [A, B, C], target_host=tgt_host, name="myadd")


In [15]:
ctx = tvm.context('llvm', 0)

n = 1024
a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), ctx)
b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype), ctx)
c = tvm.nd.array(np.zeros(n, dtype=C.dtype), ctx)
fadd(a, b, c)
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())

In [16]:
print(fadd.get_source())

 %0, %0* %9, i64 0, i32 3, i32 2
  %76 = load i16, i16* %75, align 2
  %77 = icmp eq i16 %76, 1
  %78 = and i1 %74, %77
  br i1 %78, label %assert_end33, label %assert_fail32, !prof !6

assert_fail32:                                    ; preds = %assert_end31
  %79 = load void (i8*)*, void (i8*)** @__TVMAPISetLastError, align 8, !tbaa !7
  tail call void %79(i8* getelementptr inbounds ([198 x i8], [198 x i8]* @.str.5, i64 0, i64 0))
  ret i32 -1

assert_end33:                                     ; preds = %assert_end31
  %80 = getelementptr inbounds %0, %0* %9, i64 0, i32 6
  %81 = load i64, i64* %80, align 8
  %82 = icmp eq i64 %81, 0
  br i1 %82, label %assert_end35, label %assert_fail34, !prof !6

assert_fail34:                                    ; preds = %assert_end33
  %83 = load void (i8*)*, void (i8*)** @__TVMAPISetLastError, align 8, !tbaa !7
  tail call void %83(i8* getelementptr inbounds ([116 x i8], [116 x i8]* @.str.6, i64 0, i64 0))
  ret i32 -1

assert_end35:            

In [23]:
from tvm.contrib import cc
from tvm.contrib import util

temp = util.tempdir()
fadd.save(temp.relpath("myadd.o"))
cc.create_shared(temp.relpath("myadd.so"), [temp.relpath("myadd.o")])
print(temp.listdir())

['myadd.so', 'myadd.o']


In [24]:
fadd1 = tvm.runtime.load_module(temp.relpath("myadd.so"))
fadd1(a, b, c)
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())

In [25]:
fadd.export_library(temp.relpath("myadd_pack.so"))
fadd2 = tvm.runtime.load_module(temp.relpath("myadd_pack.so"))
fadd2(a, b, c)
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())